In [ ]:
""""Data taken up from Movielens dataset.
USing Python, we would be seeing which movie is similar to STAR WARS
We'll be working with movie names and not IDs """

In [10]:
import pandas as pd

In [30]:
r_cols = ['userId', 'movieId', 'rating']
rating = pd.read_csv('E://Gaurav/ML_Dataset/movie_lens_data/rating.csv', names = r_cols, usecols = range(3))

m_cols = ['movieId','title']
movies = pd.read_csv('E://Gaurav/ML_Dataset/movie_lens_data/movie.csv', names = m_cols, usecols = range(2))

In [ ]:
movies.head()

In [19]:
rating = pd.merge(rating,movies)


In [22]:
rating.head

<bound method NDFrame.head of Empty DataFrame
Columns: [userId, movieId, rating, title]
Index: []>

In [ ]:
# using pivot table command in data frame - this will construct a user/movie rating matrix
# matrix of every movie - index - userid, columsn- movie title

movie_Ratings = rating.pivot_table(index = ['userid'],columns = ['title'],values = 'rating')

# Such type of matrix is useful for both item based as well  as user based collborative filtering

In [ ]:
# since we specifically want the users who voted for Black Sunday:

Star_Wars_rating = movie_Ratings['Star Wars (1977)']
Star_Wars_rating.head(2)

In [ ]:
# now we shall find correlation of Black_sunday movie with every other movie in movie_ratings which is very important to produce recommendations.
# Also dropping the ones with 'NaN'

similar_movies = movie_Ratings.corrwith(Star_Wars_rating)
similar_movies = similar_movies.dropna()
dataframe = pd.dataframe(similar_movies)
dataframe.head()

In [ ]:
# sort the result with similarty score

similar_movies.order(ascending = False)

## uhhhh... what was this ????? We got fairly obscure movies

In [ ]:
""""The main reason that we got suh a result is - we took into consideration the movies that were watched by just a handful of people.
Hence, we shall cosntruct a new dataframe by counting the votes each movie got along with the avg rating so as to reduce the chances of spurious results."""

moviestats = rating.groupby('title').agg({'rating': [np.size, np.mean]})
moviestats.head()

In [ ]:
""""Since our main aim is to egt rid of spurious data, so we get rid of movies rated by less than 100 people"""

popularmovies = moviestats['rating']['size'] >=100
moviestats[popularmovies].sort_values([('rating','mean')], ascending = False) [:15]

In [ ]:
""""Now we shall create our final data frame. we shall join this data with our original data frame to get the movies similar to star wars (and get the similarity score too)"""


df = moviestats[popularmovies],join(pd.Dataframe(similar_movies, columns=['similarity']))

In [ ]:
df.head()

In [ ]:
df.sort_values(['similarity'], ascedending=False) [:15]

In [ ]:
""""Conclusion: Although we got a data which looks better than previosu one, but still can be refined by taking different thresholds (I took 100 as count of rating, we can take 120 or 130 for even better results)""""""

In [ ]:
For a true Item based Recommendation system, we shall change some of teh steps done earlier.   

In [ ]:
user_Ratings = rating.pivot_table(index = ['userid'],columns = ['title'],values = 'rating')
user_Ratings.head()

In [ ]:
# Now we shall find correlation for EVERY movie, with every other movie

corr_matrix = user_Ratings.corr
corr_matrix.head()

In [ ]:
""""Again we got some spurious results of the movies wich are no highly correlated plus the ones which are correlated with themselves""""

In [ ]:
corr_matrix = user_Ratings.corr(method = 'pearson', min_periods=100)
corr_matrix.head()

In [ ]:
# now since we need to Recommend movie to people, we created a fake person for whom we need to recommend.

#added a fake id (0) , a person who liked movies like star wars, teh empire strikes back, but does like the romantic ones.

myratings = user_Ratings.loc[0].dropna()
myratings

In [ ]:
similar_Candidates = pd.Series()

for i in range(0, len(myratings.index)):
    print "Adding similarities for " + myRatings.index[i] + "..."
    #Retreiving similar movies
    sims = corr_matrix[myratings.index[i]].dropna()
    #Scaling the similarity by how well I rated this movie
    sims = sims.map(lambda x: x * myratings[i])
    # Add this score to list of similarity candidates
    similar_Candidates = similar_Candidates.append(sims)
    

In [ ]:
# sorting and printing the results:

similar_Candidates.sort_values(inplace = True, ascending = False)
print similar_Candidates.head()

In [ ]:
# Since we are geting duplicates, we shall use group by function to see the movie that is similar to more than 1 and count and get a stronf recommendation score

similar_Candidates = similar_Candidates.groupby(similar_Candidates.index).sum()

In [ ]:
similar_Candidates.sort_values(inplace = True, ascending = False)
print similar_Candidates.head()

In [ ]:
# another important thing to remember : Filter out movies that the person has already seen, coz recommending a movie that someone has already watched isnt helpful for him.

filtersimilarities = similar_Candidates.drop(myratings.index)
filtersimilarities

In [ ]:
# We can use correlations other than pearson
# Detect the outliers (some users rated huge amiunt of movie which might bring dispropotionate results)
# always take into consideration business perspective (sometimes instincts too)